# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [41]:
import pandas as pd

---
## 2. Load data

In [42]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [43]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [44]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [45]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

#### Removing Unnecessary variables

In [46]:
# Function to print columns with only one unique value
def print_single_unique_columns(df, df_name):
  single_unique_columns = df.nunique()[df.nunique() == 1].index.tolist()
  print(f"Columns with only one unique value in {df_name}:")
  print(single_unique_columns)

# Print columns with only one unique value for each dataframe
print_single_unique_columns(dec_prices, 'dec_prices')
print_single_unique_columns(df, 'df')
print_single_unique_columns(diff, 'diff')
print_single_unique_columns(jan_prices, 'jan_prices')
print_single_unique_columns(monthly_price_by_id, 'monthly_price_by_id')
print_single_unique_columns(price_df, 'price_df')


Columns with only one unique value in dec_prices:
[]
Columns with only one unique value in df:
[]
Columns with only one unique value in diff:
[]
Columns with only one unique value in jan_prices:
[]
Columns with only one unique value in monthly_price_by_id:
[]
Columns with only one unique value in price_df:
[]


In [48]:
import numpy as np

# Compute the correlation matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('churn')
corr_matrix = df[numeric_cols].corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df_reduced = df.drop(columns=to_drop)

# Include back the churn column if it exists
if 'churn' in df.columns:
  df_reduced['churn'] = df['churn']

df_reduced.head()

,id,channel_sales,cons_12m,cons_gas_12m,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,...,var_year_price_off_peak_var,var_year_price_peak_var,var_year_price_mid_peak_var,var_year_price_off_peak_fix,var_year_price_peak_fix,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,...,0.000061,2.627605e-05,0.000440,1.102785,49.550703,0.000131,4.100838e-05,9.084737e-04,2.086294,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,...,0.000005,6.089453e-04,0.000000,0.006465,0.000000,0.000003,1.217891e-03,0.000000e+00,0.009482,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,...,0.000006,2.558511e-07,0.000000,0.007662,0.000000,0.000004,9.450150e-08,0.000000e+00,0.000000,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,...,0.000005,0.000000e+00,0.000000,0.006465,0.000000,0.000003,0.000000e+00,0.000000e+00,0.000000,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,...,0.000015,3.552481e-06,0.000003,0.005429,0.001954,0.000011,2.896760e-06,4.860000e-10,0.000000,0


In [50]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14606 non-null  object        
 1   channel_sales                   14606 non-null  object        
 2   cons_12m                        14606 non-null  int64         
 3   cons_gas_12m                    14606 non-null  int64         
 4   date_activ                      14606 non-null  datetime64[ns]
 5   date_end                        14606 non-null  datetime64[ns]
 6   date_modif_prod                 14606 non-null  datetime64[ns]
 7   date_renewal                    14606 non-null  datetime64[ns]
 8   forecast_cons_12m               14606 non-null  float64       
 9   forecast_cons_year              14606 non-null  int64         
 10  forecast_discount_energy        14606 non-null  float64       
 11  fo